In [ ]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
## import postgress SQL 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-03-06 15:38:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-03-06 15:38:05 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
## Create Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [ ]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
giftcard_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True)
giftcard_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
### Drop the null values

giftcard_df = giftcard_df.dropna()

giftcard_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
### SQL SChema  review_id, customer_id, product_id,  product_parent, review_date make the SQL schema rows into integers

# giftcard_df =giftcard_df.withColumn(("customer_id", giftcard_df["customer_id"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn(("star_rating", giftcard_df["star_rating"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn("helpful_votes", giftcard_df["helpful_votes"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn("total_votes", giftcard_df["total_votes"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn(("product_parent", giftcard_df["product_parent"].cast(IntegerType()))
giftcard_df = giftcard_df.withColumn("customer_id", giftcard_df["customer_id"].cast(IntegerType()))
giftcard_df = giftcard_df.withColumn("product_parent", giftcard_df["product_parent"].cast(IntegerType()))
giftcard_df = giftcard_df.withColumn("star_rating", giftcard_df["star_rating"].cast(IntegerType()))
giftcard_df = giftcard_df.withColumn("helpful_votes", giftcard_df["helpful_votes"].cast(IntegerType()))
giftcard_df = giftcard_df.withColumn("total_votes", giftcard_df["total_votes"].cast(IntegerType()))

In [ ]:
## Transform data to SQL and change
from pyspark.sql.functions import to_date

review_id_table= giftcard_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R18RVCKGH1SSI9|   43081963|B001BM2MAC|     307809868| 2015-08-31|
|R3L4L6LW1PUOFY|   10951564|B00DZYEXPQ|      75004341| 2015-08-31|
|R2J8AWXWTDX2TF|   21143145|B00RTMUHDW|     529689027| 2015-08-31|
|R1PR37BR7G3M6A|   52782374|B00D7H8XB6|     868449945| 2015-08-31|
|R3BDDDZMZBZDPU|   24045652|B001XCWP34|      33521401| 2015-08-31|
| R8T6MO75ND212|   21751234|B004J2NBCO|     214932869| 2015-08-31|
|R2YWMQT2V11XYZ|    9109358|B00MOPAG8K|     863351797| 2015-08-31|
|R1V2HYL6OI9V39|    9967215|B003AHIK7U|     383470576| 2015-08-31|
|R3BLQBKUNXGFS4|   11234247|B006TKH2RO|     999128878| 2015-08-31|
|R17MOWJCAR9Y8Q|   12731488|B00W61M9K0|     622066861| 2015-08-31|
|R11EPG1GHOXMGB|   49861762|B00BXOGI3A|     688569009| 2015-08-31|
|R2797NKTEX5THN|    1541556|B0018RHWGE|     513803406| 2015-08

In [ ]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
## Build products Schema
products=giftcard_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001BM2MAC|Scotch Cushion Wr...|
|B00DZYEXPQ|Dust-Off Compress...|
|B00RTMUHDW|Amram Tagger Stan...|
|B00D7H8XB6|AmazonBasics 12-S...|
|B001XCWP34|Derwent Colored P...|
|B004J2NBCO|Quartet Magnetic ...|
|B00MOPAG8K|KITLEX40X2592UNV2...|
|B003AHIK7U|Bible Dry Highlig...|
|B006TKH2RO|Parker Ingenuity ...|
|B00W61M9K0| RFID Card Protector|
|B00BXOGI3A|The Original Stan...|
|B0018RHWGE|Zebra Sparky-1 WK...|
|B00IA8H2OG|E-Z Ink (TM) Comp...|
|B007TOIA66|Eric 11" Giant Si...|
|B00EANURCE|Brother MFCJ450 W...|
|B00T3Q0EOO|Clarity D613 DECT...|
|B00WN0N818|Permo 50 Feet Whi...|
|B001GAOTSW|Pilot G2 Retracta...|
|B003BNY7PW|AT&T DECT 6.0 Acc...|
|B00092RJX0|Pilot G2 Retracta...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
### Build customer table

##create customer total count
customers = giftcard_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
##### Build the vine table 
vine_table= giftcard_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R18RVCKGH1SSI9|          5|            0|          0|   N|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|
|R1PR37BR7G3M6A|          1|            2|          3|   N|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|
| R8T6MO75ND212|          5|            0|          0|   N|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|
|R11EPG1GHOXMGB|          4|            0|          0|   N|
|R2797NKTEX5THN|          5|            0|          0|   N|
|R2RUTMJS50E4WF|          5|            0|          0|   N|
|R1FZYL1HX7O4RM|          1|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/bigdata"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

## Ran it and changed my password pack

In [ ]:
### Write Dataframe for the 4x SQL Schemas 

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)